# Getting Started with NNPID SITL MuJoCo

This notebook demonstrates the basic usage of the drone simulation framework.

In [ ]:
# Add project root to path
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

## 1. MuJoCo Simulation

First, let's create a basic MuJoCo simulation and step through it.

In [ ]:
from src.core.mujoco_sim import create_simulator

# Create simulator with generic quadrotor model
sim = create_simulator(model="generic")

print(f"Timestep: {sim.timestep * 1000:.1f} ms")
print(f"Mass: {sim.mass:.2f} kg")
print(f"Gravity: {sim.gravity}")

In [ ]:
# Reset simulation
state = sim.reset(position=np.array([0.0, 0.0, 1.0]))

print(f"Initial position: {state.position}")
print(f"Initial quaternion: {state.quaternion}")

In [ ]:
# Run simulation with hover command
hover_cmd = np.array([0.5, 0.5, 0.5, 0.5])  # Approximate hover thrust

positions = []
velocities = []
times = []

for i in range(500):  # 1 second at 500 Hz
    state = sim.step(hover_cmd)
    
    positions.append(state.position.copy())
    velocities.append(state.velocity.copy())
    times.append(sim.get_time())

positions = np.array(positions)
velocities = np.array(velocities)
times = np.array(times)

In [ ]:
# Plot results
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

ax = axes[0]
ax.plot(times, positions[:, 0], label='X')
ax.plot(times, positions[:, 1], label='Y')
ax.plot(times, positions[:, 2], label='Z')
ax.set_ylabel('Position (m)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Hover Test')

ax = axes[1]
ax.plot(times, velocities[:, 0], label='Vx')
ax.plot(times, velocities[:, 1], label='Vy')
ax.plot(times, velocities[:, 2], label='Vz')
ax.set_ylabel('Velocity (m/s)')
ax.set_xlabel('Time (s)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Gymnasium Environment

Now let's use the Gymnasium-compatible environment.

In [ ]:
from src.environments.hover_env import HoverEnv, HoverEnvConfig

# Create environment
config = HoverEnvConfig(
    max_episode_steps=500,
    randomize_hover_position=True,
)
env = HoverEnv(config=config)

print(f"Observation space: {env.observation_space}")
print(f"Action space: {env.action_space}")

In [ ]:
# Run a random episode
obs, info = env.reset(seed=42)

print(f"Initial observation shape: {obs.shape}")
print(f"Target position: {info['target_position']}")

episode_reward = 0
episode_length = 0

while True:
    action = env.action_space.sample()  # Random action
    obs, reward, terminated, truncated, info = env.step(action)
    
    episode_reward += reward
    episode_length += 1
    
    if terminated or truncated:
        break

print(f"\nEpisode finished:")
print(f"  Length: {episode_length}")
print(f"  Total reward: {episode_reward:.2f}")
print(f"  Final position error: {info['position_error']:.2f} m")

In [ ]:
env.close()

## 3. PID Controller

Let's test the built-in PID controller.

In [ ]:
from src.controllers.base_controller import PIDController

# Create PID controller
pid = PIDController()

# Create simulator
sim = create_simulator(model="generic")
state = sim.reset(position=np.array([0.0, 0.0, 0.5]))

# Target position
target = np.array([0.0, 0.0, 1.5])

# Run closed-loop simulation
positions = []
times = []

dt = sim.timestep

for i in range(2500):  # 5 seconds
    # Compute control action
    action = pid.compute_action(state, target, dt)
    
    # Step simulation
    state = sim.step(action)
    
    positions.append(state.position.copy())
    times.append(sim.get_time())

positions = np.array(positions)
times = np.array(times)

In [ ]:
# Plot PID control results
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(times, positions[:, 0], label='X')
ax.plot(times, positions[:, 1], label='Y')
ax.plot(times, positions[:, 2], label='Z')
ax.axhline(y=target[2], color='k', linestyle='--', label='Target Z')

ax.set_xlabel('Time (s)')
ax.set_ylabel('Position (m)')
ax.set_title('PID Position Control')
ax.legend()
ax.grid(True, alpha=0.3)

plt.show()

print(f"Final position: {positions[-1]}")
print(f"Target position: {target}")
print(f"Final error: {np.linalg.norm(positions[-1] - target):.3f} m")

## 4. Coordinate Transforms

Demonstrating frame conversions between MuJoCo and NED.

In [ ]:
from src.utils.transforms import CoordinateTransforms, GPSTransforms
from src.utils.rotations import Rotations

# Position in MuJoCo frame (Z-up)
pos_mujoco = np.array([10.0, 5.0, 2.0])  # 10m forward, 5m left, 2m up

# Convert to NED
pos_ned = CoordinateTransforms.position_mujoco_to_ned(pos_mujoco)

print(f"MuJoCo position: {pos_mujoco}")
print(f"NED position: {pos_ned}")
print(f"  (North: {pos_ned[0]:.1f}m, East: {pos_ned[1]:.1f}m, Down: {pos_ned[2]:.1f}m)")

In [ ]:
# GPS conversion
ref_lat, ref_lon, ref_alt = 47.397742, 8.545594, 488.0  # Zurich

ned_position = np.array([100.0, 50.0, -10.0])  # 100m N, 50m E, 10m up

lat, lon, alt = GPSTransforms.ned_to_geodetic(
    ned_position, ref_lat, ref_lon, ref_alt
)

print(f"NED position: {ned_position}")
print(f"GPS coordinates:")
print(f"  Latitude:  {lat:.6f}°")
print(f"  Longitude: {lon:.6f}°")
print(f"  Altitude:  {alt:.1f} m")

## 5. Configuration System

In [ ]:
from src.config import load_config, get_config

# Load default configuration
config = get_config("default")

print("Simulation config:")
print(f"  Timestep: {config.simulation.timestep}")
print(f"  Gravity: {config.simulation.gravity}")

print("\nQuadrotor config:")
print(f"  Mass: {config.quadrotor.mass} kg")
print(f"  Max thrust/motor: {config.quadrotor.max_thrust_per_motor} N")

print("\nTraining config:")
print(f"  Algorithm: {config.training.algorithm}")
print(f"  Learning rate: {config.training.learning_rate}")
print(f"  Total timesteps: {config.training.total_timesteps:,}")

## Next Steps

1. **Train a controller**: Run `python scripts/train_controller.py --env hover`
2. **Connect to PX4**: Run `python scripts/run_sitl_sim.py` with PX4 SITL
3. **Evaluate policies**: Run `python scripts/evaluate.py model.zip --env hover`

Check the README.md for more detailed instructions.